In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_dir='data'
data_split='val'
classes = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')
n_epochs = 1
emsize = 32
batch_size = 2
margin = 3.

In [ ]:
# Parameters
n_epochs = 50
batch_size = 8
emsize = 128

In [ ]:
import torch.utils.data as data
from PIL import Image as PImage
import os
from datasets import ImageDataset, pad_to_size
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from ivis import Ivis
from sklearn.preprocessing import MinMaxScaler

from fastai.vision import *
from networks import siamese_learner

In [ ]:
from torchvision import transforms
import torch.nn.functional as F
import torch

tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats),
    lambda x: pad_to_size(x, (512, 512))
])
train_dataset = ImageDataset(
    f"{data_dir}/classy_coconut/train",
    classes,
    tfms=tfms
)
val_dataset = ImageDataset(
    f"{data_dir}/classy_coconut/val",
    classes,
    tfms=tfms
)
n_classes = len(classes)

print(len(train_dataset))
print(len(val_dataset))

In [ ]:
from datasets import SiameseImage

siamese_train_dataset = SiameseImage(train_dataset, True) # Returns pairs of images and target same/different
siamese_val_dataset = SiameseImage(val_dataset, False)

In [ ]:
from datasets import SiameseImageList
lls = SiameseImageList.from_datasets(siamese_train_dataset, siamese_val_dataset)

In [ ]:
dbunch = (lls
          .transform(get_transforms())
          .databunch("siamese", bs=batch_size, device="cuda:0", num_workers=8)
          .normalize(imagenet_stats)
         )

In [ ]:
learner = siamese_learner(dbunch, models.resnet50, emsize, margin)
learner.load("pair_model")

In [ ]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

colors = ['#ACB556',
 '#6B0519',
 '#317ADD',
 '#E89478',
 '#E2C6BB',
 '#11A545',
 '#04DA93',
 '#255C23',
 '#E1E690',
 '#E46065',
 '#C6F8A5',
 '#7300E8',
 '#903A28',
 '#89165F',
 '#24B131',
 '#D529F8',
 '#B406FA',
 '#1BFC03',
 '#C88197',
 '#E049C6',
 '#6CA73E',
 '#FC2E30',
 '#50A394',
 '#8EA6A3',
 '#E42044',
 '#EA78E5',
 '#8F6F2A',
 '#6F5BC9',
 '#E28F92',
 '#356097',
 '#BDA255',
 '#A68CDD',
 '#1A8127',
 '#E35BB1',
 '#C303BF',
 '#A0E7EE',
 '#2A1574',
 '#70D458',
 '#DBFB14',
 '#ED081D',
 '#51EECB',
 '#442A27',
 '#C84D7C',
 '#C9118E',
 '#CA9152',
 '#55287F',
 '#86DC17',
 '#9DB452',
 '#554D45',
 '#4D3C0F',
 '#F760EA',
 '#655EB4',
 '#4AC7C6',
 '#E8EE1E',
 '#9F8EC0',
 '#BF9D25',
 '#7DC9E2',
 '#241DD7',
 '#6E3114',
 '#3AE5E7',
 '#AC18DC',
 '#EEA33D',
 '#179935',
 '#F05348',
 '#387F47',
 '#78C6A6',
 '#5308BD',
 '#51A2C3',
 '#FB8106',
 '#4F675C',
 '#12BBBC',
 '#716F88',
 '#33E6BF',
 '#0B46F3',
 '#500A7C',
 '#5F4F1A',
 '#25E5F8',
 '#8C1021',
 '#7D6EF2',
 '#F3D3A0']

def plot_embeddings(embeddings, targets, classes, xlim=None, ylim=None, figsize=(30, 30)):
    plt.figure(figsize=figsize)
    for i in range(n_classes):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(classes)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), emsize))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
# train_embeddings_cl, train_labels_cl = extract_embeddings(train_loader, model)
# scaler = StandardScaler().fit(train_embeddings_cl)
# train_embeddings_tsne = TSNE(n_components=2, random_state=1337).fit_transform(scaler.transform(train_embeddings_cl))
# plot_embeddings(train_embeddings_tsne, train_labels_cl)
model = learner.model
val_embeddings_cl, val_labels_cl = extract_embeddings(val_loader, model)

In [ ]:
# scaler = StandardScaler().fit(val_embeddings_cl)
# val_embeddings_tsne = TSNE(n_components=2, random_state=1337).fit_transform(scaler.transform(val_embeddings_cl))

In [ ]:
ivis_ = Ivis(embedding_dims=2, k=15, model="maaten")
val_embeddings_ivis2 = ivis_.fit_transform(MinMaxScaler().fit_transform(val_embeddings_cl))

In [ ]:
# ivis_ = Ivis(embedding_dims=2, k=15, model="hinton")
# val_embeddings_ivis3 = ivis_.fit_transform(MinMaxScaler().fit_transform(val_embeddings_cl))

In [ ]:
# plot_embeddings(val_embeddings_tsne, val_labels_cl, classes)

In [ ]:
plot_embeddings(val_embeddings_ivis2, val_labels_cl, classes)

In [ ]:
# plot_embeddings(val_embeddings_ivis3, val_labels_cl, classes)

In [ ]:
widget_classes = ['back_button', 'browser_address', 'browser_bookmark', 'browser_tab', 'button', 'checkbox', 'close_button', 'menu', 'minimize_button', 'next_button', 'slider', 'text_field', 'text_object']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf', '#800080', '#808000', '#F08080']
widget_dataset = ImageDataset('/home/adrian/data/Widgets/train',
                            widget_classes,
    tfms=tfms)
n_classes = 13

In [ ]:
widget_loader = torch.utils.data.DataLoader(widget_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
widget_embeddings_cl, widget_labels_cl = extract_embeddings(widget_loader, model)

In [ ]:
ivis_ = Ivis(embedding_dims=2, k=5, model="maaten", batch_size=4)
widget_embeddings_ivis = ivis_.fit_transform(MinMaxScaler().fit_transform(widget_embeddings_cl))

In [ ]:
plot_embeddings(widget_embeddings_ivis, widget_labels_cl, widget_classes, figsize=(15, 15))

In [ ]:
plot_idxs = np.random.choice(np.arange(widget_embeddings_cl.shape[0]), 30, replace=False)
plot_embeddings(widget_embeddings_ivis[plot_idxs], widget_labels_cl[plot_idxs], widget_classes, figsize=(10, 10))